In [1]:
import math
import networkx as nx
import db, pathSearch

In [2]:
points = [[35.16487131336672, 136.9374775886536], [35.16378437005421, 136.9397842884064], [35.16297791885458, 136.9374775886536]]
link, length = db.getCircleRoadData(136.9374775886536, 35.16487131336672, 50)
# y1, x1, y2, x2 = pathSearch.rectangleArea(points)
# link, length = db.getRectangleRoadData(y1, x1, y2, x2, 1.25)
print(link)

[[[35.1643126, 136.9374495], [35.1647013, 136.937521]], [[35.1647013, 136.937521], [35.1650675, 136.9376184]], [[35.1650675, 136.9376184], [35.1654625, 136.9376998]], [[35.1648808, 136.9363407], [35.1647013, 136.937521]], [[35.1647013, 136.937521], [35.1645473, 136.9386028]], [[35.1652462, 136.9364031], [35.1650675, 136.9376184]], [[35.1650675, 136.9376184], [35.1649102, 136.9386832]]]


In [13]:
G = nx.Graph()
G = pathSearch.linkToGraph(Db.link, Db.length)
# pathSearch.connectGraph(G)

In [15]:
dict = nx.floyd_warshall(G)
print(dict)

{'[35.1624723, 136.9403682]': defaultdict(<function floyd_warshall_predecessor_and_distance.<locals>.<lambda>.<locals>.<lambda> at 0x000002C6F5DDD300>, {'[35.1624723, 136.9403682]': 0, '[35.1626084, 136.9392426]': 0.10343496, '[35.1657816, 136.9399919]': 0.462805365, '[35.1651538, 136.9398408]': 0.39165872, '[35.1653085, 136.9387857]': 0.488103249, '[35.1652278, 136.9393279]': 0.439002604, '[35.1634444, 136.940595]': 0.31957072200000003, '[35.1635255, 136.9400303]': 0.26745167599999997, '[35.1636073, 136.9394786]': 0.216560857, '[35.1645473, 136.9386028]': 0.40183560399999996, '[35.164399, 136.9396591]': 0.30611, '[35.1643158, 136.9402128]': 0.356881236, '[35.1642346, 136.9407811]': 0.40932411799999996, '[35.1628586, 136.9398764]': 0.191987666, '[35.1649102, 136.9386832]': 0.442844794, '[35.1647606, 136.9397392]': 0.34697212, '[35.1658611, 136.9394589]': 0.5104220939999999, '[35.1634194, 136.936091]': 0.43893549499999995, '[35.1632437, 136.9371944]': 0.336747755, '[35.1630828, 136.9382